In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# In-depth tutorial: optimisation & callbacks
In the single-cycle tutorial we looked at the propagation and inference steps that make up a forward pass, and how the loss can be backpropagated  and the detectors optimised. There's a lot of boilerplate code there, and in the tutorial we'll look at how the we can use a wrapper class to more simply optimise the detectors.

## VolumeWrapper
The `Volume` class already simplifies propagating muons through multiple layers and provides a few useful methods, however it doesn't provide all the functionality we would like for working with different detector states. The `tomopt.optimisation.wrapper.volume_wrapper.PanelVolumeWrapper` takes a `Volume` during initialisation and extends it extra methods to handle optimisation, loading, saving, and prediction.

In [2]:
import torch
from torch import Tensor, nn
import torch.nn.functional as F
import numpy as np
from tomopt.volume import PassiveLayer, PanelDetectorLayer, Volume, DetectorPanel
import matplotlib.pyplot as plt
import torch.nn.functional as F

/Users/giles/anaconda3/envs/tomopt/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DEVICE = torch.device('cpu')

In [4]:
def get_layers():
    layers = []
    lwh = Tensor([1,1,1])
    size = 0.1
    init_eff = 0.9
    init_res = 1e4
    n_panels = 4
    layers.append(PanelDetectorLayer(pos='above', lw=lwh[:2], z=1, size=2*size,
                                     panels=[DetectorPanel(res=init_res, eff=init_eff,
                                                      init_xyz=[0.25,0.25,1-(i*(2*size)/n_panels)], init_xy_span=[1.0, 1.0],
                                                      device=DEVICE) for i in range(n_panels)]))
    for z in [0.8,0.7,0.6,0.5,0.4,0.3]:
        layers.append(PassiveLayer(lw=lwh[:2], z=z, size=size, device=DEVICE))
    layers.append(PanelDetectorLayer(pos='below', lw=lwh[:2], z=0.2, size=2*size,
                                     panels=[DetectorPanel(res=init_res, eff=init_eff,
                                                      init_xyz=[0.25,0.25,0.2-(i*(2*size)/n_panels)], init_xy_span=[1.0, 1.0],
                                                      device=DEVICE) for i in range(n_panels)]))

    return nn.ModuleList(layers) 

In [5]:
volume = Volume(get_layers())
volume

Volume(
  (layers): ModuleList(
    (0): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([1.]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
        (1): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.9500]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
        (2): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.9000]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
        (3): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.8500]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
      )
    )
    (1): PassiveLayer located at z=tensor([0.8000])
    (2): PassiveLayer located at z=tensor([0.7000])
    (3): PassiveLayer located at z=tensor([0.6000])
    (4): PassiveLayer located at z=te

In [6]:
from functools import partial
from tomopt.optimisation import PanelVolumeWrapper, VoxelX0Loss

Additionally, when building the wrapper, we supply optimisers for both the resolution and efficiency, and also the loss function we want to use.

In [7]:
wrapper = PanelVolumeWrapper(volume,
                             xy_pos_opt=partial(torch.optim.SGD, lr=5e4),
                             z_pos_opt=partial(torch.optim.SGD, lr=5e3),
                             xy_span_opt=partial(torch.optim.SGD, lr=5e3),
                             loss_func=VoxelX0Loss(target_budget=20, cost_coef=None))  # Loss is precision + budget_coef*cost_coef, balance coef as required or leave as None to automatically balance on first batch

### Methods and parameters

In [8]:
wrapper.get_detectors()

[PanelDetectorLayer(
   (panels): ModuleList(
     (0): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([1.]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
     (1): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.9500]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
     (2): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.9000]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
     (3): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.8500]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
   )
 ),
 PanelDetectorLayer(
   (panels): ModuleList(
     (0): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.2000]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
     (1): <class 'tomopt.volume.p

In [9]:
wrapper.get_param_count(trainable=True)  # number of trainable parameters

40

In [10]:
wrapper.get_param_count(trainable=False)  # total number of parameters

40

The parameter states of the detectors and their optimisers can be saved and loaded via:

In [11]:
wrapper.save('save.pt')
wrapper.load('save.pt')

There's also a class-method for instantiating directly from a save (volume and optimisers still need to be passed, since only the parameters states are saved):

In [12]:
PanelVolumeWrapper.from_save('save.pt', volume=volume,
                             xy_pos_opt=partial(torch.optim.SGD, lr=1e5),
                             z_pos_opt=partial(torch.optim.SGD, lr=1e4),
                             xy_span_opt=partial(torch.optim.SGD, lr=1e5),
                             loss_func=VoxelX0Loss(target_budget=20, cost_coef=None))  # Loss is precision + budget_coef*cost_coef, balance coef as required or leave as None to automatically balance on first batch

The learning rate and momentum/$\beta_1$ of the optimisers are accessible as parameters:

In [13]:
print(wrapper.get_opt_lr('xy_pos_opt'), wrapper.get_opt_mom('z_pos_opt'), wrapper.get_opt_lr('xy_pos_opt'), wrapper.get_opt_mom('z_pos_opt'))
wrapper.set_opt_lr(1e-4, 'xy_pos_opt')
print(wrapper.get_opt_lr('xy_pos_opt'))

50000.0 0 50000.0 0
0.0001


In [14]:
wrapper.load('save.pt')

### Optimisation
As discussed in the Hello World tutorial, optimisation of the detector configuration runs over several loops: multiple batches of muons can be used to construct inference of the x0 of a given passive volume, and the loss can be accumulated over multiple different passive volumes. Additionally, there can different sets of passive volumes for training and validation.

#### Passive volumes
Passive volumes are held in `tomopt.optimisation.data.passives.PassiveYielder` objects. These iterate over a (shuffled) list of fixed passive volumes. In the future they will also be able to generate and yield a set number of randomised volumes using the `tomopt.optimisation.data.passives.PassiveGenerator` class.

In [15]:
from tomopt.core import X0

In [16]:
def arb_rad_length(*,z:float, lw:Tensor, size:float) -> Tensor:
    rad_length = torch.ones(list((lw/size).long()))*X0['beryllium']
    if z >= 0.4 and z <= 0.5: rad_length[5:,5:] = X0['lead']
    return rad_length

In [17]:
def arb_rad_length2(*,z:float, lw:Tensor, size:float) -> Tensor:
    rad_length = torch.ones(list((lw/size).long()))*X0['beryllium']
    if z >= 0.4 and z <= 0.5: rad_length[5:,5:] = X0['copper']
    return rad_length

In [18]:
from tomopt.optimisation import PassiveYielder

In [19]:
passives = PassiveYielder([arb_rad_length,arb_rad_length2], shuffle=True)

In [20]:
for p in passives: print(p)

(<function arb_rad_length2 at 0x7f7ed81c9ee0>, None)
(<function arb_rad_length at 0x7f7edddc8af0>, None)


#### Fitting loop
The fitting loop initiated via `VolumeWrapper.fit` is:

1. for epoch in `n_epochs`:
    1. `loss` = 0
    1. for `p`, `passive` in enumerate(`trn_passives`):
        1. load `passive` into passive volume
        1. for muon_batch in range(`n_mu_per_volume`//`mu_bs`):
            1. Irradiate volume with `mu_bs` muons
            1. Infer passive volume
        1. Compute loss based on precision and cost, and add to `loss`
        1. if `p`+1 % `passive_bs` == 0:
            1. `loss` = `loss`/`passive_bs`
            1. Backpropagate `loss` and update detector parameters
            1. `loss` = 0
        1. if len(`trn_passives`)-(`p`+1) < `passive_bs`:
            1. Break
            
    1. `val_loss` = 0
    1. for `p`, `passive` in enumerate(`val_passives`):
        1. load `passive` into passive volume
        1. for muon_batch in range(`n_mu_per_volume`//`mu_bs`):
            1. Irradiate volume with `mu_bs` muons
            1. Infer passive volume
            1. Compute loss based on precision and cost, and add to `val_loss`
        1. if len(`val_passives`)-(`p`+1) < `passive_bs`:
            1. Break
    1. `val_loss` = `val_loss`/`p`

I.e. the loss for a single passive can be computed over several iterations of muon irradiation (for computational speed), and the total loss can be accumulated over multiple passives before updating the detector. We also have the option of evaluating the detector on unseen passives via validation data. For this example, we'll use the same passive for the both training and validation.

In implementation, the loop is broken up into several functions:
- `_fit_epoch` runs one full epoch of volumes and updates for both training and validation
- `_scan_volumes` runs over all training/validation volumes, updating parameters when necessary
- `_scan_volume` irradiates a single volume with muons multiple batches, and computes the loss for that volume

##### Callbacks and stateful optimisation
The optimisation loop has many interjection points for callback classes. These are class which can be used to affect most aspects of the optimisation, without having to modify the code itself. Additionally, the optimisation loop is *stateful*, meaning that all aspects of the optimisation (data, losses, infrence classes, etc.) are attributes of a state class `VolumeWrapper.fit_params`. Every callback will have the `VolumeWrapper` as an attribute, and so every callback can access the `fit_params` object and so can read and modify these states. Additionally, since the callbacks are themselves an attribute of `fit_params`, every callback has access to every other callback. This carries several advantages:
- Callbacks are able to adjust their logic based on the presence and order of other callbacks.
- Callbacks can adjust their logic according to whether the optimisation loop in in "train", "valid" or "test" state (`fit_params.state`).
- Information accumulated by one callback can be used by another, without explicitly sharing it.
- Interjection calls are cleaner since no arguments need to be passed, and users aren't restricted by what the devs thought were necessary to be passed.
- Optimiser functions are cleaner, since callbacks read the majority of, what would otherwise be arguments, from the `fit_params`.

Accounting for the interjection calls (`on_*_begin` & `on_*_end`), the full optimisation loop is:
1. Associate callbacks with wrapper (`set_wrapper`)
1. `on_train_begin`
1. for epoch in `n_epochs`:
    1. `state` = "train"
    1. `on_epoch_begin`
    1. for `p`, `passive` in enumerate(`trn_passives`):
        1. if `p` % `passive_bs` == 0:
            1. `on_volume_batch_begin`
            1. `loss` = 0
        1. load `passive` into passive volume
        1. `on_volume_begin`
        1. for muon_batch in range(`n_mu_per_volume`//`mu_bs`):
            1. `on_mu_batch_begin`
            1. Irradiate volume with `mu_bs` muons
            1. Infer scatter locations
            1. `on_scatter_end`
            1. Infer x0 and append to list of x0 predictions
            1. `on_mu_batch_end`
        1. `on_x0_pred_begin`
        1. Compute overall x0 prediction
        1. `on_x0_pred_end`
        1. Compute loss based on precision and cost, and add to `loss`
        1. if `p`+1 % `passive_bs` == 0:
            1. `loss` = `loss`/`passive_bs`
            1. `on_volume_batch_end`
            1. Zero parameter gradients
            1. `on_backwards_begin`
            1. Backpropagate `loss` and compute parameter gradients
            1. `on_backwards_end`
            1. Update detector parameters
            1. Ensure detector parameters are within physical boundaries (`AbsDetectorLayer.conform_detector`)
            1. `loss` = 0
        1. if len(`trn_passives`)-(`p`+1) < `passive_bs`:
            1. Break
    1. `on_epoch_end`
    1. `state` = "valid"
    1. `on_epoch_begin`
    1. for `p`, `passive` in enumerate(`val_passives`):
        1. if `p` % `passive_bs` == 0:
            1. `on_volume_batch_begin`
            1. `loss` = 0
        1. `on_volume_begin`
        1. for muon_batch in range(`n_mu_per_volume`//`mu_bs`):
            1. `on_mu_batch_begin`
            1. Irradiate volume with `mu_bs` muons
            1. Infer scatter locations
            1. `on_scatter_end`
            1. Infer x0 and append to list of x0 predictions
            1. `on_mu_batch_end`
        1. `on_x0_pred_begin`
        1. Compute overall x0 prediction
        1. `on_x0_pred_end`
        1. Compute loss based on precision and cost, and add to `loss`
        1. if `p`+1 % `passive_bs` == 0:
            1. `loss` = `loss`/`passive_bs`
            1. `on_volume_batch_end`
        1. if len(`val_passives`)-(`p`+1) < `passive_bs`:
            1. Break
    1. `on_epoch_end`
1. `on_train_end`

Callbacks are aimed to be user-writable to modify the existing code to their specific interests and requirements. Some callbacks will be included in TomOpt, e.g. `tomopt.optimisation.callbacks.grad_callbacks.NoMoreNaNs`:

```python
class NoMoreNaNs(Callback):
    def on_backwards_end(self) -> None:
        for l in self.wrapper.volume.get_detectors():
            torch.nan_to_num_(l.resolution.grad, 0)
            torch.nan_to_num_(l.efficiency.grad, 0)
```

This only acts at one interjection point (the others are implemented in the parent `tomopt.optimisation.callbacks.callback.Callback` class as `pass`), and modifies the gradients of the volume parameters to remove `NaN` values, which sometimes occur.

Several specific subclasses of `Callback` are specifically looked for, and filtered into lists of callbacks in `VolumeWrapper.fit_params`:
- `tomopt.optimisation.callbacks.cyclic_callbacks.CyclicCallback` - callbacks that modify optimiser parameters in a cyclical manner, e.g. cosine annealing and 1cycle, and may be used to sync other callbacks to their cycle, e.g. to delay early stopping until a cycle has ended.
- `tomopt.optimisation.callbacks.eval_metric.EvalMetric` - callbacks designed to compute performance metrics
- `tomopt.optimisation.callbacks.monitors.MetricLogger` - a special callback that tracks training and validation losses and the results of the `EvalMetric`s, and displays realtime information during training.

### Fitting
Let's run the optimisation. We'll also include our set of recommended callbacks:

- `NoMoreNaNs`: Currently in TomOpt, there is a problem which means that sometimes the gradients for some parameters are `NaN` after backpropagation. The `tomopt.optimisation.callbacks.grad_callbacks.NoMoreNaNs` will automatically go through each parameter after backpropagation and replace `NaN` gradients with zeros prior to the optimiser update step.
- `PanelMetricLogger`: It is useful to have an indication of the progress the optimisation, and the current state of the detector. `tomopt.optimisation.callbacks.monitors.MetricLogger` provides real-time progress updates and telemetric data.
- `CostCoefWarmup`: The loss contains a coefficient which is used to balance the error and cost components. This can be set manually or automatically. The loss contains a method to set the coefficient automatically, but it does so using the first volume tested, and so is subject to fluctuations. `tomopt.optimisation.callbacks.loss_callbacks.CostCoefWarmup` is a callback which will run a specified number of warm-up epochs during which the detectors will not be optimised, and instead a more stable cost coefficient will be computed.
- `PanelOptConfig`: A recent addition is the `tomopt.optimisation.callbacks.opt_callbacks.PanelOptConfig` callback. Like the `CostCoefWarmup` it uses a warm-up period to track the gradients received by the detector parameters. The user can then instead specify their desired rate of change rate in physical units, and the learning rates of the optimisers will be overwritten to achieve the specified rates.
- `MuonResampler`: When optimising the detectors, we only want to consider muons which pass through the passive volume, however not all muons are guaranteed to do so. The `tomopt.optimisation.callbacks.data_callbacks.MuonResampler` will automatically regenerate muons which will not pass through the passive volume. Since this relies on "truth" information in the generator, this should not be used for realistic simulations.

In [21]:
from tomopt.optimisation import NoMoreNaNs, PanelMetricLogger, CostCoefWarmup, PanelOptConfig, MuonResampler

In [22]:
ml = PanelMetricLogger()

In [23]:
passives = PassiveYielder([arb_rad_length])

In [24]:
wrapper = PanelVolumeWrapper(volume,
                             xy_pos_opt=partial(torch.optim.SGD, lr=5e4),
                             z_pos_opt=partial(torch.optim.SGD, lr=5e3),
                             xy_span_opt=partial(torch.optim.SGD, lr=1e4),
                             loss_func=VoxelX0Loss(target_budget=20, cost_coef=None))  # Loss is precision + budget_coef*cost_coef, balance coef as required or leave as None to automatically balance on first batch

In [25]:
volume

Volume(
  (layers): ModuleList(
    (0): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([1.]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
        (1): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.9500]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
        (2): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.9000]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
        (3): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.8500]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
      )
    )
    (1): PassiveLayer located at z=tensor([0.8000])
    (2): PassiveLayer located at z=tensor([0.7000])
    (3): PassiveLayer located at z=tensor([0.6000])
    (4): PassiveLayer located at z=te

CostCoefWarmup: Freezing optimisation for 5 epochs
PanelOptConfig: Freezing optimisation for 5 epochs


/Users/giles/cernbox/mode_muon_tomography/tomopt/inference/scattering.py:339: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  idxs = torch.combinations(torch.arange(0, unc.shape[-1]), with_replacement=True)


Automatically setting cost coefficient to 0.0005572207737714052


/Users/giles/cernbox/mode_muon_tomography/tomopt/optimisation/callbacks/monitors.py:219: UserWarning: Attempting to set identical left == right == 1.0 results in singular transformations; automatically expanding.
  ax.set_xlim(1 / self.n_trn_batches, x[-1])


CostCoefWarmup: Warmed up, average error = tensor([0.0007])
PanelOptConfig: Optimiser warm-up completed
+-------------+---------------+---------------------+
|    Param    |  Median Grad  |          LR         |
+-------------+---------------+---------------------+
|  xy_pos_opt |  0.0019219768 |  5.202976435304069  |
|  z_pos_opt  |   2.1288877   | 0.00234864436934006 |
| xy_span_opt | 0.00039544993 |  25.287651707194435 |
+-------------+---------------+---------------------+


<Figure size 1024x1024 with 0 Axes>

In [26]:
_ = wrapper.fit(n_epochs=15,  # Run for 10 epochs
                passive_bs=1,  # Update every passive volumes (we only have one)
                n_mu_per_volume=1000,  # Use 1000 per passive volume
                mu_bs=100,  # Run the 1000 muons per volume in 10 batches of 100 muons (quicker than a single batch of 1000)
                trn_passives=passives,
                val_passives=passives,  # Evaluate on the training data
                cbs=[CostCoefWarmup(n_warmup=5), PanelOptConfig(n_warmup=5, xy_pos_rate=0.01, z_pos_rate=0.005, xy_span_rate=0.01), MuonResampler(), NoMoreNaNs(),ml])  # Use these three callbacks during training

In [27]:
volume.get_cost()

tensor(8.3970, grad_fn=<AddBackward0>)

In [28]:
ml.get_loss_history()

({'Training': [0.0005599763826467097,
   0.0006578947068192065,
   0.000556012790184468,
   0.0006404128507710993,
   0.0011286071967333555,
   0.0004049894050695002,
   0.0004518484929576516,
   0.0005590248620137572,
   0.000600209052208811,
   0.0004789370286744088,
   0.0003732889308594167,
   0.0006178793264552951,
   0.0003740595711860806,
   0.000534389924723655,
   0.0012381419073790312],
  'Validation': [0.0004056912730447948,
   0.00044504404650069773,
   0.000373921066056937,
   0.000557685736566782,
   0.000812061014585197,
   0.0012389726471155882,
   0.0005257811862975359,
   0.0004822208429686725,
   0.0008753618458285928,
   0.0006463592872023582,
   0.0004360128950793296,
   0.0005139713757671416,
   0.0005766533431597054,
   0.0009064764017239213,
   0.0004903679946437478]},
 {})

In [29]:
ml.get_results(loaded_best=False)

{'loss': 0.0004903679946437478}

### Prediction
Having optimised the detector, we now want to use it to predict the composition of passive volumes without updating; since we're still in simulation, we can assume that we'll always have access to the true composition.
`VolumeWrapper.predict` can be used to run the predictions for each volume in a `PassiveYielder`, and returns a list of pairs of numpy arrays; `[0]` is the prediction, and `[1]` is the truth.
The function reuses a lot of the code used during fitting, running in `fit_params.state = "test"` mode. This means that we still have the functionality to run muons in smaller batches, and the use of callbacks. Indeed, the predictions themselves are collected by a special callback `tomopt.optimisation.callbacks.pred_callbacks.PredHandler`:

```python
class PredHandler(Callback):
    r"""
    Default callback for predictions. Collects predictions for a range of volumes and returns them as list of numpy arrays
    """

    def on_pred_begin(self) -> None:
        super().__init__()
        self.preds: List[Tuple[np.ndarray, np.ndarray]] = []

    def get_preds(self) -> List[Tuple[np.ndarray, np.ndarray]]:
        return self.preds

    def on_x0_pred_end(self) -> None:
        if self.wrapper.fit_params.state == "test":
            self.preds.append((self.wrapper.fit_params.pred.detach().cpu().numpy(), self.wrapper.volume.get_rad_cube().detach().cpu().numpy()))
```

Whilst this is the default prediction callback, we are free to pass our own prediction callbacks to the prediction function as a separate argument to any other callback we wish to use. When running predictions, `on_train_*` methods are not called, instead `on_pred_begin` and `on_pred_end` are called at similar points.

As well as just running inference of the passive volume, we may wish to gather diagnostic information about the hits and scatter locations. `tomopt.optimisation.callbacks.diagnostic_callbacks` contains:
- `ScatterRecord` to record the reconstructed scatter locations in the passive volume
- `HitRecord` to record the reconstructed hit locations in the detectors

Warning: currently these are really only valid when running predictions for a single passive volume (like we will).

In [30]:
from tomopt.optimisation import PredHandler, ScatterRecord, HitRecord

In [31]:
sr = ScatterRecord()
hr = HitRecord()

In [32]:
preds = wrapper.predict(PassiveYielder([arb_rad_length]), n_mu_per_volume=10000, mu_bs=100, pred_cb=PredHandler(), cbs=[MuonResampler(), sr, hr])

#### X0 prediction
We can plot the predictions per passive volume (although we only have one) using `tomopt.plotting.predictions.plot_pred_true_x0`, which takes a single prediction and truth pair.

In [33]:
from tomopt.plotting import plot_pred_true_x0

In [34]:
plot_pred_true_x0(preds[0][0], preds[0][1])

We can see the outline of the lead block, however due to the low number of muons used, and the high uncertainty in z, the difference extends over several layers.

#### Diagnostics
Since we passed the `*Record` callbacks during prediction, they will have accumulated their relevant information which we can access by calling their `get_record` function:

In [35]:
scatter_df = sr.get_record(as_df=True)
scatter_df

,x,y,z,layer
0,0.125396,0.410562,0.553329,2
1,0.219483,0.585520,0.397501,4
2,0.141850,0.650622,0.653131,1
3,0.738045,0.756379,0.559568,2
4,0.313760,0.953669,0.657919,1
...,...,...,...,...
4019,0.928399,0.113872,0.227099,5
4020,0.417242,0.460301,0.702738,0
4021,0.891528,0.597848,0.543653,2
4022,0.590191,0.922718,0.233452,5


In [36]:
hit_df = hr.get_record(as_df=True)
hit_df

,x,y,z,layer
0,0.166108,0.697739,0.999595,0
1,0.169538,0.700246,0.956742,1
2,0.174955,0.705718,0.893787,2
3,0.180480,0.710902,0.831487,3
4,0.236325,0.760277,0.184164,4
...,...,...,...,...
76971,0.246303,0.414321,0.831487,3
76972,0.415731,0.379872,0.184164,4
76973,0.425929,0.377978,0.146512,5
76974,0.437532,0.375351,0.101858,6


We can also plot these to get the densities of hits and scatters per layer using the plotting functions in `tomopt.plotting.diagnostics`

In [37]:
from tomopt.plotting import plot_hit_density, plot_scatter_density

In [38]:
plot_scatter_density(scatter_df)

In [39]:
plot_hit_density(hit_df[(hit_df.x.abs()<5) & (hit_df.y.abs()<5)])